# Core-to-log matching

## Contents

* [Problem description](#Problem-description)
* [Well matching](#Well-matching)
* [Dataset matching](#Dataset-matching)
* [Matching research](#Matching-research)
* [Conclusion](#Conclusion)

## Problem description

Perform core-to-log matching by shifting core samples in order to maximize correlation between well and core logs.

Shifted core samples must satisfy the following constraints:
* Boring intervals constraints:
    * boring intervals must be shifted by no more than 10 meters
    * boring intervals must not overlap
    * the order of boring intervals must remain unchanged
    * if several boring intervals are extracted one after another, they must be shifted by the same delta
* Lithology intervals constraints (if defined for a well):
    * lithology intervals can be moved only inside the corresponding boring interval
    * lithology intervals must not overlap
    * the order of lithology intervals must remain unchanged

In [1]:
import os
import sys

sys.path.insert(0, os.path.join("..", "..", ".."))
from petroflow import Well, WellDataset
from petroflow.batchflow import Pipeline, C, L
from petroflow.batchflow.research import Research, Option

The following matching mode precedence (from highest to lowest) is used in all experiments. Each mode is specified as follows:

`<well_log> ~ <sign><core_attr>.<core_log>`, where:
* `well_log` - mnemonic of a well log to use
* `sign` - a sign of the theoretical correlation between well and core data
* `core_attr` - an attribute of a well to get core data from
* `core_log` - mnemonic of a core log or property to use

In [2]:
MATCHING_MODES = [
    "GK ~ +core_logs.GK",
    "GGKP ~ +core_logs.DENSITY",
    "GGKP ~ -core_properties.POROSITY",
    "GK ~ -core_properties.POROSITY",
]

## Well matching

If you need to process only a single well, specify the path to it and create a `Well` instance:

In [3]:
WELL_PATH = "/path/to/a/well/"
well = Well(path=WELL_PATH)

First, let's visualize how much the well and core data are shifted relative to each other. Since the matching is not performed yet, you need to explicitly specify which core and well logs you want to compare. In this case, we will take a look at gamma ray logs:

In [4]:
well.plot_matching(mode="GK ~ +core_logs.GK")

The figure shows that coring intervals must be shifted down for about two meters to perfectly match the well data. Now let's run the matching procedure and look at how the well will change:

In [5]:
well = well.check_regularity().match_core_logs(mode=MATCHING_MODES)
well.plot_matching()

As we can see, the the GR log of the core is now almost identical to the GR log of the well. Also note, that three last boring sequences were connected to each other to increase matching $R^{2}$.

## Dataset matching

Usually, we want to match a group of wells (e.g. all wells from a field) instead of a single well. `PetroFlow` provides a simple way to do this in just several lines of code. First, we need to specify a path to the wells we want to match and create a dataset:

In [6]:
DATASET_PATH = "/path/to/a/dataset/*"
well_ds = WellDataset(path=DATASET_PATH, dirs=True, sort=True)

Matched wells will be saved in the `MATCHED_DATASET_PATH` directory:

In [7]:
MATCHED_DATASET_PATH = "/path/to/a/matched/dataset/"

Matching pipeline consists of 3 actions:
* check, that well data is consistent
* perform core-to-log matching with the given modes and save a matching report for each well
* dump matched wells in the `MATCHED_DATASET_PATH` directory

In [8]:
matching_pipeline = (Pipeline()
    .check_regularity()
    .match_core_logs(mode=MATCHING_MODES, save_report=True)
    .dump(MATCHED_DATASET_PATH)
    .run(batch_size=1, n_epochs=1, shuffle=False, drop_last=False, bar=True, lazy=True)
)

Now we need to link the created dataset to the pipeline and run it:

In [9]:
(well_ds >> matching_pipeline).run()

100%|██████████| 15/15 [01:18<00:00,  5.22s/it]


As a result, `MATCHED_DATASET_PATH` will contain directories with all matched wells.

## Matching research

Sometimes we need to run a pipeline several times with different parameters. For example, we may want to smooth the logs with Gaussian filters of different sizes before performing core-to-log matching. To do this, we will use the `research` module. As in the previous case, first, we need to create a dataset of wells:

In [10]:
DATASET_PATH = "/path/to/a/dataset/*"
well_ds = WellDataset(path=DATASET_PATH, dirs=True, sort=True)

Then we specify parameters of our pipeline: a directory to save the results and a list of Gaussian kernel sizes:

In [11]:
MATCHED_DATASETS_PATH = "/path/to/a/matched/dataset/"
grid = Option("win_size", [None, 101, 201, 301])

Now we create a matching pipeline, that is almost identical to the one in the previous section except for several arguments, that it gets from the config of the `Research`:

In [12]:
matching_pipeline = (Pipeline()
    .check_regularity()
    .match_core_logs(mode=MATCHING_MODES, gaussian_win_size=C("win_size"))
    .dump(L(os.path.join)(MATCHED_DATASETS_PATH, C("win_size").str()))
    .run(batch_size=4, n_epochs=1, shuffle=False, drop_last=False, lazy=True)
)

Finally, we create a `Research` instance, specify the dataset, the pipeline and its parameters and run it:

In [13]:
research = (Research()
    .add_pipeline(matching_pipeline, dataset=well_ds, name="matching")
    .add_grid(grid)
)

research.run(name="matching_research", bar=True)

Research matching_research is starting...
Distributor has 4 jobs


100%|██████████| 4/4 [05:32<00:00, 83.03s/it] 


As a result, `MATCHED_DATASETS_PATH` will contain directories for each Gaussian kernel size with matched wells inside.

## Conclusion

In this notebook, we learned how to use `PetroFlow` to:
* perform core-to-log matching for a single well
* match the whole dataset of wells at once
* run the matching pipeline several times with different parameters